In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import duckdb
import pandas as pd
import numpy as np
import os

In [ ]:
duck_con = duckdb.connect('tripadvisor.db')
user_visits = duck_con.execute(
    """SELECT userid, COUNT(0) total_visits
                    FROM clickstream2
                 WHERE visit_tripadvisor = TRUE
                 GROUP BY userid"""
).df()
user_visits.describe()

In [ ]:
users_time_between_visits = duck_con.execute("""WITH prebase_first_visit AS (
                        SELECT userid, eventdate, date_casted, event_order
                        FROM clickstream2
                        WHERE visit_tripadvisor = TRUE
                        AND day_visited_number = 1
                    ), second_visit AS (
                        SELECT userid, eventdate, date_casted, event_order, day_visited_number
                        FROM clickstream2
                        WHERE clickstream2.visit_tripadvisor = TRUE
                        AND day_visited_number = 2
                    )
                 SELECT prebase_first_visit.userid, DATEDIFF('day', prebase_first_visit.date_casted, second_visit.date_casted) AS days_between_visits
                    FROM prebase_first_visit
                    INNER JOIN second_visit ON prebase_first_visit.userid = second_visit.userid"""
).df()
users_time_between_visits

In [ ]:
# show the cumulative percent of users from day 1 to day 30 
all_days = users_time_between_visits['days_between_visits'].value_counts().sort_index().reset_index()
all_days.columns = ['days_between_visits', 'count']
all_days['cumulative_percent'] = round((all_days['count'].cumsum()/len(users_time_between_visits))*100, 2)

only_30_days = all_days[all_days['days_between_visits'] <= 30]

# plot with plotly the cumulative percent of users from day 1 to day 30
# Add a annotation only when the cumulative percent is 80%

fig = px.line(
    only_30_days,
    x="days_between_visits",
    y="cumulative_percent",
    title="Cumulative percent of users from day 1 to day 30",
)
# Set fig axis Y from 0 to 100 and X from 0 to 30
fig.update_yaxes(range=[0, 100])
fig.update_xaxes(range=[0, 30])

fig.add_trace(
    go.Scatter(
        x=only_30_days["days_between_visits"],
        y=only_30_days["cumulative_percent"]
    )
)
users_80_percent = only_30_days[only_30_days["cumulative_percent"] >= 80]
x_value = users_80_percent["days_between_visits"].min()
y_value = users_80_percent["cumulative_percent"].min()
fig.add_annotation(
    x=x_value,
    y=y_value,
    text=f"80% of users have returned in {x_value} days",
    showarrow=True,
    arrowhead=1,
)
fig.show()

In [ ]:
days_between_before_first_visit = duck_con.execute("""
WITH prebase_first_visit AS (
    SELECT userid,
        date_casted,
        event_order
    FROM clickstream2
    WHERE visit_tripadvisor = TRUE
        AND day_visited_number = 1
)
SELECT first_event.userid,
    first_event.date_casted first_date_event,
    prebase_first_visit.date_casted first_date_visit,
    DATE_DIFF('day', first_event.date_casted, prebase_first_visit.date_casted) days_between
FROM clickstream2 first_event
INNER JOIN prebase_first_visit ON first_event.userid = prebase_first_visit.userid
WHERE first_event.event_order = 1"""
).df()

# show the cumulative percent of users from day 1 to day 30 
all_days = days_between_before_first_visit['days_between'].value_counts().sort_index().reset_index()
all_days.columns = ['days_between', 'count']
all_days['cumulative_percent'] = round((all_days['count'].cumsum()/len(days_between_before_first_visit))*100, 2)

until_n_day = 60
only_30_days = all_days[all_days['days_between'] <= until_n_day]

# plot with plotly the cumulative percent of users from day 1 to day until_n_day
# Add a annotation only when the cumulative percent is 80%

fig = px.line(
    only_30_days,
    x="days_between",
    y="cumulative_percent",
    title="Cumulative percent of users from day 1 to day 30",
)
# Set fig axis Y from 0 to 100 and X from 0 to 30
fig.update_yaxes(range=[0, 100])
fig.update_xaxes(range=[0, until_n_day])

fig.add_trace(
    go.Scatter(
        x=only_30_days["days_between"],
        y=only_30_days["cumulative_percent"]
    )
)
users_80_percent = only_30_days[only_30_days["cumulative_percent"] >= 20]
x_value = users_80_percent["days_between"].min()
y_value = users_80_percent["cumulative_percent"].min()
fig.add_annotation(
    x=x_value,
    y=y_value,
    text=f"20% of users have first visit in {x_value} days",
    showarrow=True,
    arrowhead=1,
)
fig.show()

In [ ]:
days_between_before_first_visit_and_last_session = duck_con.execute("""
WITH prebase_first_visit AS (
    SELECT userid,
        date_casted,
        event_order
    FROM clickstream2
    WHERE visit_tripadvisor = TRUE
        AND day_visited_number = 1
),
previous_session AS (
SELECT clickstream2.userid,
    MAX(clickstream2.date_casted) previous_date_casted
FROM clickstream2
INNER JOIN prebase_first_visit ON clickstream2.userid = prebase_first_visit.userid
WHERE clickstream2.date_casted < prebase_first_visit.date_casted
GROUP BY clickstream2.userid
)
SELECT prebase_first_visit.userid,
    previous_session.previous_date_casted,
    prebase_first_visit.date_casted first_date_visit,
    DATE_DIFF('day', previous_session.previous_date_casted, prebase_first_visit.date_casted) days_between
FROM prebase_first_visit 
INNER JOIN previous_session ON previous_session.userid = prebase_first_visit.userid"""
).df()

# show the cumulative percent of users from day 1 to day 30 
all_days = days_between_before_first_visit_and_last_session['days_between'].value_counts().sort_index().reset_index()
all_days.columns = ['days_between', 'count']
all_days['cumulative_percent'] = round((all_days['count'].cumsum()/len(days_between_before_first_visit_and_last_session))*100, 2)

until_n_day = 60
only_30_days = all_days[all_days['days_between'] <= until_n_day]

# plot with plotly the cumulative percent of users from day 1 to day until_n_day
# Add a annotation only when the cumulative percent is 80%

fig = px.line(
    only_30_days,
    x="days_between",
    y="cumulative_percent",
    title="How many days pass from the first visit to the previous day session in traffic?",
)
# Set fig axis Y from 0 to 100 and X from 0 to 30
fig.update_yaxes(range=[0, 100])
fig.update_xaxes(range=[0, until_n_day])

fig.add_trace(
    go.Scatter(
        x=only_30_days["days_between"],
        y=only_30_days["cumulative_percent"]
    )
)
# Set percent of 2 standard deviation
users_80_percent = only_30_days[only_30_days["cumulative_percent"] >= 95]
x_value = users_80_percent["days_between"].min()
y_value = users_80_percent["cumulative_percent"].min()
fig.add_annotation(
    x=x_value,
    y=y_value,
    text=f"95% of users have first visit in {x_value} days",
    showarrow=True,
    arrowhead=1,
)
fig.show()

In [ ]:
# Get all events of users before the first visit event
users_events_before_first_visit = duck_con.execute(
    """WITH prebase_first_visit AS (
                        SELECT userid, eventdate, date_casted, event_order,
                            (date_casted - '5 days'::interval) AS session_before
                        FROM clickstream2
                        WHERE visit_tripadvisor = TRUE
                        AND visit_number = 1
                        ORDER BY eventdate
                        LIMIT 25
                    )
                    SELECT clickstream2.userid,
                        clickstream2.eventdate,
                        clickstream2.eventtimestamp,
                        clickstream2.date_casted,
                        clickstream2.referrerurl,
                        clickstream2.targeturl,
                        clickstream2.event_order
                    FROM clickstream2
                    INNER JOIN prebase_first_visit ON clickstream2.userid = prebase_first_visit.userid
                    WHERE clickstream2.event_order < prebase_first_visit.event_order
                    AND clickstream2.eventdate >= prebase_first_visit.session_before
                    AND clickstream2.userid = '111a0458-bd4b-4781-ba21-2875fee04b86'
                    ORDER BY clickstream2.userid, clickstream2.event_order"""
).df()
users_events_before_first_visit

In [ ]:
test = duck_con.execute(
    """WITH prebase_first_visit AS (
    SELECT DISTINCT userid,
        eventdate,
        date_casted,
        event_order,
        (date_casted - '1 month'::interval) AS one_month_before
    FROM clickstream2
    WHERE visit_tripadvisor = TRUE
        AND visit_number = 1
    ORDER BY eventdate
    LIMIT 25
),
data_before_first_visit AS (
    SELECT clickstream2.userid,
        clickstream2.date_casted,
        clickstream2.event_order,
        clickstream2.referrerurl previous_url,
        clickstream2.targeturl current_url,
        LEAD(clickstream2.targeturl) OVER (
            PARTITION BY clickstream2.userid 
            ORDER BY clickstream2.event_order
        ) AS next_target_url
    FROM clickstream2
    INNER JOIN prebase_first_visit ON clickstream2.userid = prebase_first_visit.userid
    WHERE clickstream2.event_order < prebase_first_visit.event_order
    AND clickstream2.eventdate >= prebase_first_visit.one_month_before
),
filtered_data AS (
    SELECT userid,
           date_casted,
           event_order,
           previous_url,
           current_url
    FROM data_before_first_visit
    WHERE current_url != next_target_url OR next_target_url IS NULL
)

SELECT userid,
       date_casted,
       current_url AS url,
       ROW_NUMBER() OVER (PARTITION BY userid ORDER BY event_order) AS sequence
FROM filtered_data
ORDER BY userid, sequence;
"""
).df()
test

In [ ]:
duck_con.execute("""DROP TABLE user_sequence""")

In [ ]:
duck_con.execute("""CREATE TABLE IF NOT EXISTS user_sequence AS WITH prebase_first_visit AS (
    SELECT DISTINCT userid,
        eventdate,
        date_casted,
        event_order,
        (date_casted - '5 days'::interval) AS session_before
    FROM clickstream2
    WHERE visit_tripadvisor = TRUE
        AND visit_number = 1
        AND have_problem_with_timestamp = FALSE
    ORDER BY eventdate
),
data_before_first_visit AS (
    SELECT clickstream2.userid,
        clickstream2.date_casted,
        clickstream2.event_order,
        clickstream2.referrerurl previous_url,
        clickstream2.targeturl current_url,
        LEAD(clickstream2.targeturl) OVER (
            PARTITION BY clickstream2.userid
            ORDER BY clickstream2.event_order
        ) AS next_target_url
    FROM clickstream2
        INNER JOIN prebase_first_visit ON clickstream2.userid = prebase_first_visit.userid
    WHERE clickstream2.event_order < prebase_first_visit.event_order
        AND clickstream2.eventdate >= prebase_first_visit.session_before
        AND clickstream2.have_problem_with_timestamp = FALSE
),
filtered_data AS (
    SELECT userid,
        date_casted,
        event_order,
        previous_url,
        current_url
    FROM data_before_first_visit
    WHERE current_url != next_target_url
        OR next_target_url IS NULL
)
SELECT userid,
    date_casted,
    current_url AS url,
    ROW_NUMBER() OVER (
        PARTITION BY userid
        ORDER BY event_order
    ) AS sequence,
    ROW_NUMBER() OVER (
        PARTITION BY userid
        ORDER BY event_order DESC
    ) AS reverse_sequence
FROM filtered_data
ORDER BY userid,
    sequence;"""
)

In [ ]:
duck_con.execute("""SELECT userid,
    event_order,
    date_casted,
    referrerurl,
    regexp_extract(referrerurl, '^(?:https?:\/\/)?(?:[^@\/\n]+@)?(?:www\.)?([^:\/\n]+)', 1) AS previous_domain,
    targeturl,
    regexp_extract(targeturl, '^(?:https?:\/\/)?(?:[^@\/\n]+@)?(?:www\.)?([^:\/\n]+)', 1) AS current_domain
FROM clickstream2
LIMIT 10;"""
).df()

In [ ]:
duck_con.execute("""DROP TABLE user_sequence_domains""")

In [ ]:
duck_con.execute("""CREATE TABLE IF NOT EXISTS user_sequence_domains AS WITH prebase_first_visit AS (
    SELECT DISTINCT userid,
        eventdate,
        date_casted,
        event_order,
        (date_casted - '5 days'::interval) AS session_before
    FROM clickstream2
    WHERE visit_tripadvisor = TRUE
        AND visit_number = 1
        AND clickstream2.have_problem_with_timestamp = FALSE
    ORDER BY eventdate
),
domain_extraction AS (
    SELECT clickstream2.userid,
        clickstream2.event_order,
        clickstream2.date_casted,
        regexp_extract(referrerurl, '^(?:https?:\/\/)?(?:[^@\/\n]+@)?(?:www\.)?([^:\/\n]+)', 1) AS previous_domain,
        regexp_extract(targeturl, '^(?:https?:\/\/)?(?:[^@\/\n]+@)?(?:www\.)?([^:\/\n]+)', 1) AS current_domain
    FROM clickstream2
        INNER JOIN prebase_first_visit p ON clickstream2.userid = p.userid
    WHERE clickstream2.event_order < p.event_order
        AND clickstream2.date_casted >= p.session_before
        AND clickstream2.have_problem_with_timestamp = FALSE
),
data_before_first_visit AS (
    SELECT d.userid,
        d.date_casted,
        d.event_order,
        d.previous_domain,
        d.current_domain,
        LEAD(d.current_domain) OVER (
            PARTITION BY d.userid
            ORDER BY d.event_order
        ) AS next_domain
    FROM domain_extraction d
),
filtered_data AS (
    SELECT userid,
        date_casted,
        event_order,
        previous_domain,
        current_domain
    FROM data_before_first_visit
    WHERE (
        current_domain != next_domain
        AND current_domain IS NOT NULL
        AND current_domain != ''
    )
    OR next_domain IS NULL
)
SELECT userid,
    date_casted,
    current_domain AS domain,
    ROW_NUMBER() OVER (
        PARTITION BY userid
        ORDER BY event_order
    ) AS sequence,
    ROW_NUMBER() OVER (
        PARTITION BY userid
        ORDER BY event_order DESC
    ) AS reverse_sequence
FROM filtered_data
ORDER BY userid,
    sequence"""
)

In [ ]:
test_user_id = '111a0458-bd4b-4781-ba21-2875fee04b86'

In [ ]:
check_user_sequence = duck_con.execute(f"""SELECT userid,
    sequence,
    reverse_sequence,
    date_casted,
    domain
FROM user_sequence_domains
WHERE userid = '{test_user_id}'
ORDER BY sequence;""").df()
check_user_sequence

In [ ]:
check_user_sequence = duck_con.execute(f"""SELECT userid,
    sequence,
    reverse_sequence,
    date_casted,
    url
FROM user_sequence
WHERE userid = '{test_user_id}'
ORDER BY sequence;""").df()
check_user_sequence

## Top urls visited before tripadvisor

In [ ]:
duck_con.execute("""SELECT url, COUNT(0) as count 
FROM user_sequence
GROUP BY url
ORDER BY count DESC
LIMIT 10;
""").df()

In [ ]:
duck_con.execute("""SELECT domain, COUNT(0) as count 
FROM user_sequence_domains
GROUP BY domain
ORDER BY count DESC
LIMIT 250;
""").df()[['domain']].to_csv('domains.csv', index=False)

In [ ]:
websites_categories = {
    "Search Engines": ["google.com", "bing.com", "duckduckgo.com", "yahoo.com", "ecosia.org"],
    "Social Media": ["facebook.com", "instagram.com", "twitter.com", "tumblr.com", "linkedin.com", "pinterest.com", "tiktok.com", "messenger.com", "discord.com", "whatsapp.com", "reddit.com", "snapchat.com"],
    "Streaming & Entertainment": ["youtube.com", "netflix.com", "hulu.com", "hbomax.com", "spotify.com", "vimeo.com"],
    "E-commerce": ["amazon.com", "ebay.com", "walmart.com", "etsy.com", "aliexpress.com", "bestbuy.com", "lowes.com", "kohls.com", "macys.com", "hotels.com", "rakuten.com", "wayfair.com", "gap.com", "shein.com"],
    "News & Media": ["nytimes.com", "cnn.com", "foxnews.com", "washingtonpost.com", "wsj.com", "usatoday.com", "bbc.com", "cnbc.com", "nypost.com", "forbes.com", "theguardian.com", "businessinsider.com", "buzzfeed.com"],
    "Technology & Web Services": ["microsoft.com", "apple.com", "github.com", "adobe.com", "slack.com", "salesforce.com", "dropbox.com", "godaddy.com", "cloudfront.net", "wix.com"],
    "Education": ["wikipedia.org", "instructure.com", "quizlet.com", "blackboard.com", "chegg.com", "harvard.edu", "ucdavis.edu", "usc.edu", "stanford.edu", "rutgers.edu", "umich.edu", "stanford.edu", "columbia.edu", "cuny.edu", "ufl.edu", "wisc.edu", "ucf.edu", "pitt.edu", "washington.edu", "duke.edu", "yale.edu", "northwestern.edu", "miami.edu", "wustl.edu", "utexas.edu", "nyu.edu", "arizona.edu", "studentdoctor.net", "squarespace.com", "ucla.edu", "upenn.edu", "umn.edu"],
    "Health": ["nih.gov", "mayoclinic.org", "webmd.com", "cdc.gov"],
    "Finance & Banking": ["chase.com", "bankofamerica.com", "paypal.com", "americanexpress.com", "fidelity.com", "wellsfargo.com", "capitalone.com", "intuit.com", "citi.com", "vanguard.com", "schwab.com", "robinhood.com"],
    "Business & Productivity": ["office.com", "zoom.us", "slack.com", "salesforce.com", "glassdoor.com"],
    "Ads & Marketing": ["doubleclick.net", "googlesyndication.com", "googleadservices.com"],
    "Online Communities & Forums": ["quora.com", "fandom.com", "medium.com", "stackexchange.com", "reddit.com", "studentdoctor.net"],
    "Travel & Accommodation": ["expedia.com", "airbnb.com", "booking.com", "priceline.com", "kayak.com", "flyfrontier.com", "delta.com"],
    "Government": ["irs.gov", "state.gov", "uscis.gov", "va.gov"],
}


# Create a table with the categories
duck_con.execute("""DROP TABLE IF EXISTS websites_categories""")
duck_con.execute("""DROP SEQUENCE IF EXISTS website_id""")

duck_con.sql("""CREATE SEQUENCE website_id START 1;""")

duck_con.execute("""CREATE TABLE IF NOT EXISTS websites_categories (
    id INT DEFAULT NEXTVAL('website_id'),
    domain VARCHAR(255) NOT NULL,
    category VARCHAR(255) NOT NULL
);
""")

for category, domains in websites_categories.items():
    for domain in domains:
        duck_con.execute(f"""INSERT INTO websites_categories (domain, category) VALUES ('{domain}', '{category}');""")

In [ ]:
duck_con.execute("""WITH prebase AS (
    SELECT domain,
        COUNT(0) as total_events
    FROM user_sequence_domains
    GROUP BY domain
    ORDER BY 2 DESC
    LIMIT 1000
)
SELECT domain,
    CASE
        WHEN LENGTH(domain) - LENGTH(REPLACE(domain, '.', '')) >= 2 THEN
            CONCAT(
                REVERSE(SUBSTRING(
                    REVERSE(domain),
                    position('.' in REVERSE(domain)) + 1,
                    position('.' in SUBSTRING(REVERSE(domain), position('.' in REVERSE(domain)) + 1)) -1 
                )),
                REVERSE(SUBSTRING(
                    REVERSE(domain),
                    1,
                    position('.' in REVERSE(domain))
                ))
            )
        ELSE domain
    END AS base_domain
FROM prebase
WHERE LENGTH(domain) - LENGTH(REPLACE(domain, '.', '')) >= 2
ORDER BY total_events DESC;
""").df()

In [ ]:
duck_con.execute("""UPDATE user_sequence_domains
SET domain = CASE
        WHEN LENGTH(domain) - LENGTH(REPLACE(domain, '.', '')) >= 2 THEN
            CONCAT(
                REVERSE(SUBSTRING(
                    REVERSE(domain),
                    position('.' in REVERSE(domain)) + 1,
                    position('.' in SUBSTRING(REVERSE(domain), position('.' in REVERSE(domain)) + 1)) -1 
                )),
                REVERSE(SUBSTRING(
                    REVERSE(domain),
                    1,
                    position('.' in REVERSE(domain))
                ))
            )
        ELSE domain
    END
WHERE LENGTH(domain) - LENGTH(REPLACE(domain, '.', '')) >= 2;
""")

In [ ]:
parent_treemap = duck_con.execute("""
WITH prebase_domains_categories AS (
    SELECT user_sequence_domains.domain, 
        CASE WHEN category IS NULL THEN 'Other' ELSE category END AS category,
        COUNT(0) as total_events
FROM user_sequence_domains
LEFT JOIN websites_categories ON user_sequence_domains.domain = websites_categories.domain
GROUP BY user_sequence_domains.domain, 2
)
SELECT category, CAST(SUM(total_events) AS INT) as total_events
FROM prebase_domains_categories
GROUP BY category
ORDER BY total_events DESC
LIMIT 50;
""").df()

child_treemap = duck_con.execute("""
WITH prebase_domains_categories AS (
    SELECT user_sequence_domains.domain,
        CASE WHEN category IS NULL THEN 'Other' ELSE category END AS category,
        COUNT(0) as total_events
FROM user_sequence_domains
LEFT JOIN websites_categories ON user_sequence_domains.domain = websites_categories.domain
GROUP BY user_sequence_domains.domain, 2
)
SELECT category, domain, CAST(total_events AS INT) as total_events
FROM prebase_domains_categories
ORDER BY total_events DESC
LIMIT 50;
""").df()

In [ ]:
# # create a treemap with plotly
fig = px.treemap(
    child_treemap,
    path=['category', 'domain'],
    values='total_events',
    color='total_events',
    color_continuous_scale='RdBu',
    title='Treemap of domains',
    hover_data=['total_events'],
    color_continuous_midpoint=np.average(child_treemap['total_events'], weights=child_treemap['total_events'])
)

fig.show()

## Top urls visited as last touch before tripadvisor

In [ ]:
duck_con.execute("""SELECT url, COUNT(0) as count 
FROM user_sequence
WHERE reverse_sequence = 1
GROUP BY url
ORDER BY count DESC
LIMIT 10;
""").df()

In [ ]:
duck_con.execute("""SELECT domain, COUNT(0) as count 
FROM user_sequence_domains
WHERE reverse_sequence = 1
GROUP BY domain
ORDER BY count DESC
LIMIT 10;
""").df()

In [ ]:
duck_con.execute("""SELECT *
FROM user_sequence
WHERE url like '%tripadvisor%'
ORDER BY userid, sequence
LIMIT 10;
""").df()

## Size of journeys to tripadvisor

In [ ]:
journeys = duck_con.execute(
    """SELECT userid,
    COUNT(0) AS total_steps
FROM user_sequence
GROUP BY userid;"""
).df()
journeys.describe()

In [ ]:
domain_journeys = duck_con.execute(
    """SELECT userid,
    COUNT(0) AS total_steps
FROM user_sequence_domains
GROUP BY userid;"""
).df()
domain_journeys.describe()

## Top last 10 steps journey

In [ ]:
journey_last_10_steps = duck_con.execute("""SELECT *
FROM user_sequence_domains
WHERE reverse_sequence <= 10
ORDER BY userid, sequence;"""
).df()

journey_last_10_steps

In [ ]:
journey_last_10_steps[journey_last_10_steps['domain'] == '']

In [ ]:
journey_last_10_steps.pivot(
    index='userid', columns='reverse_sequence', values='domain'
).reset_index().info()

In [ ]:
# convert dataframe from row to column getting values using reverse_sequence as column name and values as domain
journey = journey_last_10_steps.pivot(
    index='userid', columns='reverse_sequence', values='domain'
).reset_index()
journey.columns = ['userid', 'step_10', 'step_9', 'step_8', 'step_7', 'step_6', 'step_5', 'step_4', 'step_3', 'step_2', 'step_1']
journey

# convert journey dataframe to get top 10 most common journeys
unique_journeys = dict()
for index, row in journey.iterrows():
    only_steps = row[1:]
    only_steps = only_steps.values.tolist()
    only_steps = tuple(only_steps)

    if only_steps in unique_journeys:
        unique_journeys[only_steps] += 1
    else:
        unique_journeys[only_steps] = 1

unique_journeys = pd.DataFrame.from_dict(unique_journeys, orient='index').reset_index()
unique_journeys.columns = ['journey', 'count']
unique_journeys = unique_journeys.sort_values(by='count', ascending=False)
unique_journeys.head(10)

In [ ]:
# plot top 10 most common journeys using plotly and sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=journey.columns[1:],
        color="blue"
    ),
    link=dict(
        source=[0, 0, 0, 0, 0, 0, 0, 0, 0],
        target=[1, 2, 3, 4, 5, 6, 7, 8, 9],
        value=unique_journeys['count']
    )
)])

fig.update_layout(title_text="Top 10 most common journeys")
fig.show()